# Creating Coastal Watershed

In [1]:
# imports
### Reqiured Imports###
import pandas, os
import arcpy
from matplotlib import pyplot as plt

In [2]:
# define the path to coastal wetlands shapefile
inDCW = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Coastalwetland\shapefiles"

# Coastal Wetlands
wetlands_avg_inun = os.path.join(inDCW, "Wetland_connected_avg_inundation_NAD1983")
wetlands_high_inun = os.path.join(inDCW, "wetlands_connected_high_inundation_NAD1983")
wetlands_low_inun = os.path.join(inDCW, "wetlands_connected_low_inundation_NAD1983")
wetlands_surge = os.path.join(inDCW, "wetlands_connected_surge_inundation_NAD1983")

# define the path to streams 
inStreams = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\GLB_Stream\GLB_stream_Ras_FeatureToLine.shp"



### 1- Buffer the streams
Buffer Polyline to create your riverin riperian area

In [3]:
# Buffer inStreams by 50 meters 
outBuffer = r"D:\Users\abolmaal\code\CoastalWatersheds\GLB_Stream\Buffer_50m\GLB_stream_Ras_FeatureToLine_Buffer_50m.shp"

# buffer inStreams by 50 meters
arcpy.Buffer_analysis(inStreams, outBuffer, "50 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\Users\\abolmaal\\code\\CoastalWatersheds\\GLB_Stream\\Buffer_50m\\GLB_stream_Ras_FeatureToLine_Buffer_50m.shp'>

### 2- Erase your coastal wetlands that overlap with your riperian area

In [4]:
outErase_path = r"D:\Users\abolmaal\code\CoastalWatersheds\Erase_Riperian"
erase_buffer_avg = os.path.join(outErase_path, "Wetland_connected_avg_erasebuff_50.shp")
erase_buffer_high = os.path.join(outErase_path, "Wetland_connected_high_erasebuff_50.shp")
erase_buffer_low = os.path.join(outErase_path, "Wetland_connected_low_erasebuff_50.shp")
erase_buffer_surge = os.path.join(outErase_path, "Wetland_connected_surge_erasebuff_50.shp")

# Erase the outBuffer from the wetlands_avg_inun shapefile with the output name erase_buffer_avg
arcpy.Erase_analysis(wetlands_avg_inun, outBuffer, erase_buffer_avg, None)

# Erase the outBuffer from the wetlands_high_inun shapefile with the output name erase_buffer_high
arcpy.Erase_analysis(wetlands_high_inun, outBuffer, erase_buffer_high, None)

# Erase the outBuffer from the wetlands_low_inun shapefile with the output name erase_buffer_low
arcpy.Erase_analysis(wetlands_low_inun, outBuffer, erase_buffer_low, None)

# Erase the outBuffer from the wetlands_surge shapefile with the output name erase_buffer_surge
arcpy.Erase_analysis(wetlands_surge, outBuffer, erase_buffer_surge, None)

<Result 'D:\\Users\\abolmaal\\code\\CoastalWatersheds\\Erase_Riperian\\Wetland_connected_surge_erasebuff_50.shp'>

#### 2-1 Add a field called coastal_Id

In [5]:
# add a field called coastal_id to the erase_buffer_avg shapefile that is equal to FID field +1
arcpy.AddField_management(erase_buffer_avg, "coastal_Id", "LONG")
arcpy.CalculateField_management(erase_buffer_avg, "coastal_id", "!FID! + 1", "PYTHON3")

# add a field called coastal_id to the erase_buffer_high shapefile that is equal to FID field +1
arcpy.AddField_management(erase_buffer_high, "coastal_Id", "LONG")
arcpy.CalculateField_management(erase_buffer_high, "coastal_id", "!FID! + 1", "PYTHON3")

# add a field called coastal_id to the erase_buffer_low shapefile that is equal to FID field +1
arcpy.AddField_management(erase_buffer_low, "coastal_Id", "LONG")
arcpy.CalculateField_management(erase_buffer_low, "coastal_id", "!FID! + 1", "PYTHON3")

# add a field called coastal_id to the erase_buffer_surge shapefile that is equal to FID field +1
arcpy.AddField_management(erase_buffer_surge, "coastal_Id", "LONG")
arcpy.CalculateField_management(erase_buffer_surge, "coastal_id", "!FID! + 1", "PYTHON3")


<Result 'D:\\Users\\abolmaal\\code\\CoastalWatersheds\\Erase_Riperian\\Wetland_connected_surge_erasebuff_50.shp'>

### 3- Create a watershed

convert erase_buffer_avg polygon to raster with the output name erase_buffer_avg_raster

In [6]:
# convert erase_buffer_avg polygon to raster with the output name erase_buffer_avg_raster
erase_buffer_avg_raster = os.path.join(outErase_path, "Wetland_connected_avg_raster.tif")
arcpy.PolygonToRaster_conversion(erase_buffer_avg, "coastal_Id", erase_buffer_avg_raster, "CELL_CENTER", "NONE", "50")

# convert erase_buffer_high polygon to raster with the output name erase_buffer_high_raster
erase_buffer_high_raster = os.path.join(outErase_path, "Wetland_connected_high_raster.tif")
arcpy.PolygonToRaster_conversion(erase_buffer_high, "coastal_Id", erase_buffer_high_raster, "CELL_CENTER", "NONE", "50")

# convert erase_buffer_low polygon to raster with the output name erase_buffer_low_raster
erase_buffer_low_raster = os.path.join(outErase_path, "Wetland_connected_low_raster.tif")
arcpy.PolygonToRaster_conversion(erase_buffer_low, "coastal_Id", erase_buffer_low_raster, "CELL_CENTER", "NONE", "50")

## convert erase_buffer_surge polygon to raster with the output name erase_buffer_low_raster
erase_buffer_surge_raster = os.path.join(outErase_path, "Wetland_connected_surge_raster.tif")
arcpy.PolygonToRaster_conversion(erase_buffer_surge, "coastal_Id", erase_buffer_surge_raster, "CELL_CENTER", "NONE", "50")



<Result 'D:\\Users\\abolmaal\\code\\CoastalWatersheds\\Erase_Riperian\\Wetland_connected_surge_raster.tif'>

Create Watershed

In [8]:
D8_flow  = r"S:\Projects\Active\GLB_Nutrient_Transport\DEM_rasters\GLB_Bdry_buff10km_dem_fill_dir.tif"

# Create watershed for erase_buffer_avg_raster with the output name erase_buffer_avg_watershed and the flow direction raster D8_flow and set then snap raster and extent to D8_flow
erase_buffer_avg_watershed = os.path.join(outErase_path, "Wetland_connected_avg_watershed.tif")
arcpy.gp.Watershed_sa(D8_flow, erase_buffer_avg_raster, erase_buffer_avg_watershed)

# Create watershed for erase_buffer_high_raster with the output name erase_buffer_high_watershed and the flow direction raster D8_flow and set then snap raster and extent to D8_flow
erase_buffer_high_watershed = os.path.join(outErase_path, "Wetland_connected_high_watershed.tif")
arcpy.gp.Watershed_sa(D8_flow, erase_buffer_high_raster, erase_buffer_high_watershed)

# Create watershed for erase_buffer_low_raster with the output name erase_buffer_low_watershed and the flow direction raster D8_flow and set then snap raster and extent to D8_flow
erase_buffer_low_watershed = os.path.join(outErase_path, "Wetland_connected_low_watershed.tif")
arcpy.gp.Watershed_sa(D8_flow, erase_buffer_low_raster, erase_buffer_low_watershed)

# Create watershed for erase_buffer_surge_raster with the output name erase_buffer_surge_watershed and the flow direction raster D8_flow and set then snap raster and extent to D8_flow
erase_buffer_surge_watershed = os.path.join(outErase_path, "Wetland_connected_surge_watershed.tif")
arcpy.gp.Watershed_sa(D8_flow, erase_buffer_surge_raster, erase_buffer_surge_watershed)



<geoprocessing server result object at 0x1a04ea74270>

In [9]:
# convert erase_buffer_avg_watershed to polygon with the output name erase_buffer_avg_watershed_poly
erase_buffer_avg_watershed_poly = os.path.join(outErase_path, "Wetland_connected_avg_watershed.shp")
arcpy.RasterToPolygon_conversion(erase_buffer_avg_watershed, erase_buffer_avg_watershed_poly, "SIMPLIFY", "VALUE")

# convert erase_buffer_high_watershed to polygon with the output name erase_buffer_high_watershed_poly
erase_buffer_high_watershed_poly = os.path.join(outErase_path, "Wetland_connected_high_watershed.shp")
arcpy.RasterToPolygon_conversion(erase_buffer_high_watershed, erase_buffer_high_watershed_poly, "SIMPLIFY", "VALUE")

# convert erase_buffer_low_watershed to polygon with the output name erase_buffer_low_watershed_poly
erase_buffer_low_watershed_poly = os.path.join(outErase_path, "Wetland_connected_low_watershed.shp")
arcpy.RasterToPolygon_conversion(erase_buffer_low_watershed, erase_buffer_low_watershed_poly, "SIMPLIFY", "VALUE")

# convert erase_buffer_surge_watershed to polygon with the output name erase_buffer_surge_watershed_poly
erase_buffer_surge_watershed_poly = os.path.join(outErase_path, "Wetland_connected_surge_watershed.shp")
arcpy.RasterToPolygon_conversion(erase_buffer_surge_watershed, erase_buffer_surge_watershed_poly, "SIMPLIFY", "VALUE")

<Result 'D:\\Users\\abolmaal\\code\\CoastalWatersheds\\Erase_Riperian\\Wetland_connected_surge_watershed.shp'>

### 4- Erase Lake drainage watersed from coastal watershed

In [10]:
outpath = r"D:\Users\abolmaal\code\CoastalWatersheds\Erase_drainage"
path_to_streamwatershed = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Streamwatershed\PointWaterdhed_LH.shp"


# Erase the erase_buffer_avg_watershed_poly from the path_to_streamwatershed shapefile with the output name CoastalWatershed_avg_erase_lakedrain.shp and add a field called coastal_id to the output shapefile that is equal to FID+1

CoastalWatershed_avg_erase_lakedrain = os.path.join(outpath, "CoastalWatershed_avg_erase_lakedrain.shp")
arcpy.Erase_analysis(erase_buffer_avg_watershed_poly,path_to_streamwatershed, CoastalWatershed_avg_erase_lakedrain, None)
arcpy.CalculateField_management(CoastalWatershed_avg_erase_lakedrain, "coastal_id", "!FID! + 1", "PYTHON3")

# Erase the erase_buffer_high_watershed_poly from the path_to_streamwatershed shapefile with the output name CoastalWatershed_high_erase_lakedrain.shp and add a field called coastal_id to the output shapefile that is equal to FID +1
CoastaWatershed_high_erase_lakedrain = os.path.join(outpath, "CoastalWatershed_high_erase_lakedrain.shp")
arcpy.Erase_analysis(erase_buffer_high_watershed_poly, path_to_streamwatershed, CoastaWatershed_high_erase_lakedrain, None)
arcpy.CalculateField_management(CoastaWatershed_high_erase_lakedrain, "coastal_id", "!FID! + 1", "PYTHON3")


# Erase the erase_buffer_low_watershed_poly from the path_to_streamwatershed shapefile with the output name CoastalWatershed_low_erase_lakedrain.shp and add a field called coastal_id to the output shapefile that is equal to FID +1
CoastaWatershed_low_erase_lakedrain = os.path.join(outpath, "CoastalWatershed_low_erase_lakedrain.shp")
arcpy.Erase_analysis(erase_buffer_low_watershed_poly, path_to_streamwatershed, CoastaWatershed_low_erase_lakedrain, None)
arcpy.CalculateField_management(CoastaWatershed_low_erase_lakedrain, "coastal_id", "!FID! + 1", "PYTHON3")

# Erase the erase_buffer_surge_watershed_poly from the path_to_streamwatershed shapefile with the output name CoastalWatershed_surge_erase_lakedrain.shp and add a field called coastal_id to the output shapefile that is equal to FID +1
CoastaWatershed_surge_erase_lakedrain = os.path.join(outpath, "CoastalWatershed_surge_erase_lakedrain.shp")
arcpy.Erase_analysis(erase_buffer_surge_watershed_poly, path_to_streamwatershed, CoastaWatershed_surge_erase_lakedrain, None)
arcpy.CalculateField_management(CoastaWatershed_surge_erase_lakedrain, "coastal_id", "!FID! + 1", "PYTHON3")



<Result 'D:\\Users\\abolmaal\\code\\CoastalWatersheds\\Erase_drainage\\CoastalWatershed_surge_erase_lakedrain.shp'>

In [12]:
# Add a field called Shape_Area to the CoastalWatershed_avg_erase_lakedrain shapefile that is equal to the area of each feature
arcpy.AddField_management(CoastalWatershed_avg_erase_lakedrain, "Shape_Area", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastalWatershed_avg_erase_lakedrain, [["Shape_Area", "AREA"]], area_unit="SQUARE_METERS")

# Add a field called Shape_Area to the CoastaWatershed_high_erase_lakedrain shapefile that is equal to the area of each feature
arcpy.AddField_management(CoastaWatershed_high_erase_lakedrain, "Shape_Area", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastaWatershed_high_erase_lakedrain, [["Shape_Area", "AREA"]], area_unit="SQUARE_METERS")

# Add a field called Shape_Area to the CoastaWatershed_low_erase_lakedrain shapefile that is equal to the area of each feature
arcpy.AddField_management(CoastaWatershed_low_erase_lakedrain, "Shape_Area", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastaWatershed_low_erase_lakedrain, [["Shape_Area", "AREA"]], area_unit="SQUARE_METERS")

# Add a field called Shape_Area to the CoastaWatershed_surge_erase_lakedrain shapefile that is equal to the area of each feature
arcpy.AddField_management(CoastaWatershed_surge_erase_lakedrain, "Shape_Area", "DOUBLE")
arcpy.CalculateGeometryAttributes_management(CoastaWatershed_surge_erase_lakedrain, [["Shape_Area", "AREA"]], area_unit="SQUARE_METERS")


<Result 'D:\\Users\\abolmaal\\code\\CoastalWatersheds\\Erase_drainage\\CoastalWatershed_surge_erase_lakedrain.shp'>

Some parts of coastal watershed are in the lake, we need to Earse the parts that are inside the Lake

In [13]:
# path to Lake Huron shapefile
Lake_Huron = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\hydro_p_LakeHuron\hydro_p_LakeHuron_84.shp"
outErase_Lake_path = r"D:\Users\abolmaal\code\CoastalWatersheds\Erase_lake"

# Erase the CoastalWatershed_avg_erase_lakedrain shapefile from the Lake_Huron shapefile with the output name CoastalWatershed_avg_erase_lakedrain_LakeHuron.shp 
# and save it in outErase_Lake_path
CoastalWatershed_avg_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake_path, "CoastalWatershed_avg_erase_lakedrain_LakeHuron.shp")
arcpy.Erase_analysis(CoastalWatershed_avg_erase_lakedrain, Lake_Huron, CoastalWatershed_avg_erase_lakedrain_LakeHuron, None)


# Erase the CoastaWatershed_high_erase_lakedrain shapefile from the Lake_Huron shapefile with the output name CoastalWatershed_high_erase_lakedrain_LakeHuron.shp
# and save it in outErase_Lake_path
CoastalWatershed_high_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake_path, "CoastalWatershed_high_erase_lakedrain_LakeHuron.shp")
arcpy.Erase_analysis(CoastaWatershed_high_erase_lakedrain, Lake_Huron, CoastalWatershed_high_erase_lakedrain_LakeHuron, None)

# Erase the CoastaWatershed_low_erase_lakedrain shapefile from the Lake_Huron shapefile with the output name CoastalWatershed_low_erase_lakedrain_LakeHuron.shp
# and save it in outErase_Lake_path
CoastalWatershed_low_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake_path, "CoastalWatershed_low_erase_lakedrain_LakeHuron.shp")
arcpy.Erase_analysis(CoastaWatershed_low_erase_lakedrain, Lake_Huron, CoastalWatershed_low_erase_lakedrain_LakeHuron, None)

# Erase the CoastaWatershed_surge_erase_lakedrain shapefile from the Lake_Huron shapefile with the output name CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp
# and save it in outErase_Lake_path
CoastalWatershed_surge_erase_lakedrain_LakeHuron = os.path.join(outErase_Lake_path, "CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp")
arcpy.Erase_analysis(CoastaWatershed_surge_erase_lakedrain, Lake_Huron, CoastalWatershed_surge_erase_lakedrain_LakeHuron, None)


<Result 'D:\\Users\\abolmaal\\code\\CoastalWatersheds\\Erase_lake\\CoastalWatershed_surge_erase_lakedrain_LakeHuron.shp'>